In [1]:
!git clone https://github.com/ZichengXu/Decoding-Tree-Sketching.git
!pip install transformers==4.54.0
%cd Decoding-Tree-Sketching

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, set_seed

import sys, os
sys.path.append(os.path.abspath('..'))

from decoding_tree_sketching.kvbatch_decoder import KVBatchEGDT
from decoding_tree_sketching.utils.eval_utils import extract_answer_llm, extract_answer_qwq

Cloning into 'Decoding-Tree-Sketching'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 159 (delta 51), reused 66 (delta 35), pack-reused 74 (from 1)
Receiving objects: 100% (159/159), 1.13 MiB | 19.62 MiB/s, done.
Resolving deltas: 100% (74/74), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 101.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
/content/Decoding-Tree-Sketching


In [2]:
# print(torch.cuda.is_available())
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Decoding hyperparameters
DECODE_CONFIG = {
    "entropy_threshold": 2.5,
    "branch_top_k": 3,
    "max_active_hyps": 12,
    "max_new_tokens": 5000,
    "temperature": 0.6,
}
tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda" if torch.cuda.is_available() else torch.device("cpu"),
    torch_dtype="auto",
    trust_remote_code=True
)
streamer = TextStreamer(tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [5]:
examples = [
  "Six points $A, B, C, D, E,$ and $F$ lie in a straight line in that order. Suppose that $G$ is a point not on the line and that $AC=26, BD=22, CE=31, DF=33, AF=73, CG=40,$ and $DG=30.$ Find the area of $\\triangle BGE.$",
]
groundtruths = [
    "468",
]
reasoning_tail = r" Please reason step by step, and put your final answer within \boxed{}."
seed = 1

set_seed(seed)
ques_idx = 0
example = examples[ques_idx]
groundtruth = groundtruths[ques_idx]
full_prompt = example + reasoning_tail
text = tokenizer.apply_chat_template(
    [{"role": "user", "content": full_prompt}],
    tokenize=False,
    add_generation_prompt=True
)

In [10]:
# Regular output

inputs = tokenizer(text, return_tensors="pt").to(model.device)
out = model.generate(
    **inputs,
    max_new_tokens=DECODE_CONFIG["max_new_tokens"],
    do_sample=True,
    temperature=DECODE_CONFIG["temperature"],
    streamer=streamer,
)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜begin▁of▁sentence｜><｜begin▁of▁sentence｜><｜User｜>Six points $A, B, C, D, E,$ and $F$ lie in a straight line in that order. Suppose that $G$ is a point not on the line and that $AC=26, BD=22, CE=31, DF=33, AF=73, CG=40,$ and $DG=30.$ Find the area of $\triangle BGE.$ Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>
Okay, so I have this problem where there are six points, A, B, C, D, E, and F, lying on a straight line in that order. There's another point G not on the line, and I need to find the area of triangle BGE. The given lengths are AC = 26, BD = 22, CE = 31, DF = 33, AF = 73, CG = 40, and DG = 30. Hmm, that seems a bit complex, but let's break it down step by step.

First, I think I should try to visualize the points on a straight line. Let me denote the positions of these points on a number line. Let's assign coordinates to each point such that the line is the x-axis. Let me choose point A as the origin, so A is at (0, 0). Then, let's de

In [16]:
num_new_tokens = out[0].shape[0] - inputs["input_ids"].shape[1]
stat = tokenizer.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
# print(gen)
ans = extract_answer_llm(stat)
print(f"Groundtruth = {groundtruth}, Regular decoding output = {ans}")

Groundtruth = 468, Regular decoding output = 24


In [17]:
kvegdt = KVBatchEGDT(model, tokenizer, seed=seed)
dts_out = kvegdt.generate(
        text,
        entropy_threshold=DECODE_CONFIG["entropy_threshold"],
        branch_top_k=DECODE_CONFIG["branch_top_k"],
        max_active_hyps=DECODE_CONFIG["max_active_hyps"],
        max_new_tokens=DECODE_CONFIG["max_new_tokens"],
        temperature=DECODE_CONFIG["temperature"],
    )

Decoding:  70%|██████▉   | 3494/5000 [02:22<01:01, 24.60it/s]


In [18]:
# print(f"*** MODEL OUTPUT ***\n{dts_out['text']}")
# Print generation statistics such as steps, branch events, and sequence length
print(f"\n*** GENERATION STATS ***\n{dts_out['stats']}")
dts_ans = extract_answer_llm(dts_out['text'])
print(f"Groundtruth = {groundtruth}, DTS output = {dts_ans}")


*** GENERATION STATS ***
{'num_steps': 3495, 'num_finished': 1, 'generated_len': 3495, 'mean_entropy_best': 0.18470470317954035, 'branch_events': 0, 'total_branches_created': 0, 'max_active_batch': 1, 'unfinished_left': 0}
Groundtruth = 468, DTS output = 468.
